In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/WI_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1040127103,WI,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14
1,1299787308,WI,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14
2,1525827110,WI,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14
3,2572157107,WI,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14
4,3943447203,WI,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1040127103,WI,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14
1,1299787308,WI,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14
2,1525827110,WI,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14
3,2572157107,WI,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14
4,3943447203,WI,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
360,5096417006,WI,"PO Box 205 N4690 BANGOR INDUSTRIAL PKWY, WEST ...",43.890741,-91.014470,NaN
547,4648077103,WI,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,NaN
1641,2925978305,WI,"215 Front St, Clinton, WI",42.410926,-71.689619,NaN
1643,7736337004,WI,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,NaN
1712,6987067006,WI,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,NaN
2027,9739697001,WI,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,NaN
2151,9170817005,WI,"7213 HWY 41 0.0, Caledonia, WI",25.763107,-80.312898,NaN
2656,2688497106,WI,"12571 OLIVER AVE S STE 100, MILWAUKEE, WI",44.776233,-93.305704,NaN
3242,5637407007,WI,"320 10th Ave., PO Box 96, BALDWIN, WI",44.966632,-92.374358,NaN
7304,5078077802,WI,"640 Fox Street, ARCADIA, WI",44.252307,-91.487735,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('55')] #WI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/19 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

19


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12546/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
360,5096417006,WI,"PO Box 205 N4690 BANGOR INDUSTRIAL PKWY, WEST ...",43.890741,-91.014470,[550630101014071]
547,4648077103,WI,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,[]
1641,2925978305,WI,"215 Front St, Clinton, WI",42.410926,-71.689619,[]
1643,7736337004,WI,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,[]
1712,6987067006,WI,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,[]
2027,9739697001,WI,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,[]
2151,9170817005,WI,"7213 HWY 41 0.0, Caledonia, WI",25.763107,-80.312898,[]
2656,2688497106,WI,"12571 OLIVER AVE S STE 100, MILWAUKEE, WI",44.776233,-93.305704,[]
3242,5637407007,WI,"320 10th Ave., PO Box 96, BALDWIN, WI",44.966632,-92.374358,[551091208012023]
7304,5078077802,WI,"640 Fox Street, ARCADIA, WI",44.252307,-91.487735,[551211006001042]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12546/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
360,5096417006,WI,"PO Box 205 N4690 BANGOR INDUSTRIAL PKWY, WEST ...",43.890741,-91.014470,550630101014071
547,4648077103,WI,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,NaN
1641,2925978305,WI,"215 Front St, Clinton, WI",42.410926,-71.689619,NaN
1643,7736337004,WI,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,NaN
1712,6987067006,WI,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,NaN
2027,9739697001,WI,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,NaN
2151,9170817005,WI,"7213 HWY 41 0.0, Caledonia, WI",25.763107,-80.312898,NaN
2656,2688497106,WI,"12571 OLIVER AVE S STE 100, MILWAUKEE, WI",44.776233,-93.305704,NaN
3242,5637407007,WI,"320 10th Ave., PO Box 96, BALDWIN, WI",44.966632,-92.374358,551091208012023
7304,5078077802,WI,"640 Fox Street, ARCADIA, WI",44.252307,-91.487735,551211006001042


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
547,4648077103,WI,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,NaN
1641,2925978305,WI,"215 Front St, Clinton, WI",42.410926,-71.689619,NaN
1643,7736337004,WI,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,NaN
1712,6987067006,WI,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,NaN
2027,9739697001,WI,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,NaN
2151,9170817005,WI,"7213 HWY 41 0.0, Caledonia, WI",25.763107,-80.312898,NaN
2656,2688497106,WI,"12571 OLIVER AVE S STE 100, MILWAUKEE, WI",44.776233,-93.305704,NaN
8669,4612047005,WI,"5074 HWY 51 NORTH, HARSHAW, WI",34.398029,-89.927004,NaN
10297,4769537008,WI,"600 North First Street, CAMERON, WI",37.346986,-121.898560,NaN
12619,6978328300,WI,"578 Allen St, Amherst, WI",42.091848,-72.543053,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
360,5096417006,550630101014071
547,4648077103,NaN
1641,2925978305,NaN
1643,7736337004,NaN
1712,6987067006,NaN
2027,9739697001,NaN
2151,9170817005,NaN
2656,2688497106,NaN
3242,5637407007,551091208012023
7304,5078077802,551211006001042


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/WI_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,942696,1040127103,WI,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14
1,942697,1299787308,WI,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14
2,942698,1525827110,WI,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14
3,942699,2572157107,WI,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14
4,942700,3943447203,WI,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,942696,1040127103,WI,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14,NaN
1,942697,1299787308,WI,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14,NaN
2,942698,1525827110,WI,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14,NaN
3,942699,2572157107,WI,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14,NaN
4,942700,3943447203,WI,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14,NaN
...,...,...,...,...,...,...,...,...
17341,960037,9282518304,WI,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,5.513320e+14,NaN
17342,960038,9361087700,WI,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,5.510100e+14,NaN
17343,960039,9411997102,WI,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,5.507909e+14,NaN
17344,960040,9453378305,WI,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,5.503397e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,942696,1040127103,WI,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,5.502501e+14,NaN,550250114051048.0
1,942697,1299787308,WI,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,5.508701e+14,NaN,550870101001029.0
2,942698,1525827110,WI,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,5.506301e+14,NaN,550630105001009.0
3,942699,2572157107,WI,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,5.508701e+14,NaN,550870126042003.0
4,942700,3943447203,WI,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,5.511100e+14,NaN,551110001031144.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1040127103,WI,"5100 Eastpark Blvd, Suite 210 0.0, Madison, WI",43.144787,-89.294690,550250114051048.0
1,1299787308,WI,"221 W. COLLEGE AVE Floor 2, APPLETON, WI",44.261498,-88.408348,550870101001029.0
2,1525827110,WI,"2700 National Drive Ste 100, ONALASKA, WI",43.879726,-91.189984,550630105001009.0
3,2572157107,WI,"W6390 Challenger Dr, Suite 203 0.0, Appleton, WI",44.260292,-88.511351,550870126042003.0
4,3943447203,WI,"10889 E PENNY LN, BARABOO, WI",43.522722,-89.780254,551110001031144.0
...,...,...,...,...,...,...
17341,9282518304,WI,"W229N5065 Duplainville Rd, Pewaukee, WI",43.109408,-88.203886,551332033044000.0
17342,9361087700,WI,"3806 DOUGLAS AVENUE, RACINE, WI",42.767675,-87.803579,551010014013000.0
17343,9411997102,WI,"6101 W VLIET ST, WAUWATOSA, WI",43.050101,-87.988944,550790912001000.0
17344,9453378305,WI,"E5690 County Road D, Menomonie, WI",44.968805,-91.875247,550339703002036.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
547,4648077103,WI,CT Corporation System Inc 1010 Dale Street Nor...,44.971659,-93.126014,NaN
1641,2925978305,WI,"215 Front St, Clinton, WI",42.410926,-71.689619,NaN
1643,7736337004,WI,"215 FRONT ST, CLINTON, WI",42.410926,-71.689619,NaN
1712,6987067006,WI,"300 RIVERSIDE DR, GILMAN, WI",40.799780,-73.971387,NaN
2027,9739697001,WI,"5231 US Hwy 41, FRANKSVILLE, WI",34.938278,-85.139313,NaN
2151,9170817005,WI,"7213 HWY 41 0.0, Caledonia, WI",25.763107,-80.312898,NaN
2656,2688497106,WI,"12571 OLIVER AVE S STE 100, MILWAUKEE, WI",44.776233,-93.305704,NaN
8669,4612047005,WI,"5074 HWY 51 NORTH, HARSHAW, WI",34.398029,-89.927004,NaN
10297,4769537008,WI,"600 North First Street, CAMERON, WI",37.346986,-121.898560,NaN
12619,6978328300,WI,"578 Allen St, Amherst, WI",42.091848,-72.543053,NaN


Convert to File

In [19]:
fips_WI = fips_merge_drop

In [20]:
fips_WI.to_csv("../../../data/state_data/geo/geo_fips/23/WI_fips_scraped.csv") 